HOLD OUT

In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.optimizers import Adam
import pandas as pd
import numpy as np

def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    train_data = data[data['Data'] < pd.to_datetime("2020-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2020-01-01")]

    y_train = train_data['Dif_Preco_Dolar']
    X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar'])

    y_test = test_data['Dif_Preco_Dolar']
    X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real', 'Dif_Preco_Dolar'])

    mean_values_train = X_train.mean()
    X_train.fillna(mean_values_train, inplace=True)

    mean_values_test = X_test.mean()
    X_test.fillna(mean_values_test, inplace=True)


    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()

    resultados = pd.DataFrame()
    resultados['Data'] = test_data['Data'].reset_index(drop=True)
    resultados['Valor_Real'] = y_test.reset_index(drop=True)
    resultados['Valor_Previsto'] = y_pred.flatten() 

    data_especifica = pd.to_datetime("2023-09-27")
    previsao_especifica = resultados[resultados['Data'] == data_especifica]
    if not previsao_especifica.empty:
        print(previsao_especifica)
    else:
        print(f"Não há dados disponíveis para a data {data_especifica.date()}")

9/9 [==============================] - 0s 950us/step
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 2.798991665323847
MAE: 1.349502540914024
R^2: -0.009438888607367968

          Data  Valor_Real  Valor_Previsto
273 2023-09-27       -1.16        0.192991
12/12 [==============================] - 0s 740us/step
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 2.761115048953175
MAE: 1.3284029350722033
R^2: -0.0014322303803457892

          Data  Valor_Real  Valor_Previsto
348 2023-09-27       -1.16        0.150049
8/8 [==============================] - 0s 947us/step
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 66.4023388782247
MAE: 6.794788465214591
R^2: -24.44573641986611

          Data  Valor_Real  Valor_Previsto
224 2023-09-27       -1.16        8.187501
12/12 [==============================] - 0s 767us/step
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv:
MSE: 5010

JANELA DESLIZANTE DE 1 DIA

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.optimizers import Adam
import pandas as pd
import numpy as np

def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

def create_sliding_window(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size), :])
        y.append(data[i + window_size, 0])
    return np.array(X), np.array(y)

file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

window_size = 1 

for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    train_data = data[data['Data'] < pd.to_datetime("2020-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2020-01-01")]

    y_train = train_data['Dif_Preco_Dolar']
    X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real','Dif_Preco_Dolar'])

    y_test = test_data['Dif_Preco_Dolar']
    X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real','Dif_Preco_Dolar'])

    mean_values_train = X_train.mean()
    X_train.fillna(mean_values_train, inplace=True)

    mean_values_test = X_test.mean()
    X_test.fillna(mean_values_test, inplace=True)

    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"MSE: {mse}, MAE: {mae}, R^2: {r2}")

    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()

    resultados = pd.DataFrame()
    resultados['Data'] = test_data['Data'].reset_index(drop=True) 
    resultados['Valor_Real'] = y_test.reset_index(drop=True) 
    resultados['Valor_Previsto'] = y_pred  

    data_especifica = pd.to_datetime("2023-09-27")
    previsao_especifica = resultados[resultados['Data'] == data_especifica]

 
    if not previsao_especifica.empty:
        print(previsao_especifica)
    else:
        print(f"Não há dados disponíveis para a data {data_especifica.date()}")


9/9 [==============================] - 0s 873us/step
MSE: 2.819625234167038, MAE: 1.355276980028335, R^2: -0.016880256532507687
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 2.819625234167038
MAE: 1.355276980028335
R^2: -0.016880256532507687

          Data  Valor_Real  Valor_Previsto
273 2023-09-27       -1.16         0.24756
12/12 [==============================] - 0s 725us/step
MSE: 2.763458098516628, MAE: 1.3293861848198247, R^2: -0.002282033922979876
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 2.763458098516628
MAE: 1.3293861848198247
R^2: -0.002282033922979876

          Data  Valor_Real  Valor_Previsto
348 2023-09-27       -1.16        0.166532
8/8 [==============================] - 0s 850us/step
MSE: 103.14540550436305, MAE: 9.32241825780298, R^2: -38.52587884287487
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 103.14540550436305
MAE: 9.32241825780298
R^2: -38.52587884287487

 

JANELA DESLIZANTE DE 30 DIAS

In [38]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.optimizers import Adam
import pandas as pd
import numpy as np


def remove_outliers(df, column_names):
    for column in column_names:
        Q1 = df[column].quantile(0.20)
        Q3 = df[column].quantile(0.80)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df

# Função para criar janela deslizante
def create_sliding_window(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:(i + window_size), :])
        y.append(data[i + window_size, 0])
    return np.array(X), np.array(y)

file_paths = ['C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv', 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv', 'C:/Users/milen/OneDrive/Documentos/TCC/Bases/taxa_br.csv', 
              'C:/Users/milen/OneDrive/Documentos/TCC/Bases/bolsa_eua.csv']

window_size = 30  # Por exemplo, 10 dias

for file_path in file_paths:
    data = pd.read_csv(file_path)

    data['Data'] = pd.to_datetime(data['Data'])
    data = data.iloc[1:]

    numeric_columns = data.select_dtypes(include=[np.number]).columns.tolist()
    data = remove_outliers(data, numeric_columns)

    train_data = data[data['Data'] < pd.to_datetime("2020-01-01")]
    test_data = data[data['Data'] >= pd.to_datetime("2020-01-01")]

    y_train = train_data['Dif_Preco_Dolar']
    X_train = train_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real','Dif_Preco_Dolar'])

    y_test = test_data['Dif_Preco_Dolar']
    X_test = test_data.drop(columns=['Data', 'Preco_Dolar', 'Preco_Real', 'Dif_Preco_Real','Dif_Preco_Dolar'])

    mean_values_train = X_train.mean()
    X_train.fillna(mean_values_train, inplace=True)

    mean_values_test = X_test.mean()
    X_test.fillna(mean_values_test, inplace=True)

    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(1, X_train.shape[2])))
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

    model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

    y_pred = model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"MSE: {mse}, MAE: {mae}, R^2: {r2}")
    print(f"Resultados para {file_path}:")
    print("MSE:", mse)
    print("MAE:", mae)
    print("R^2:", r2)
    print()

    resultados = pd.DataFrame()
    resultados['Data'] = test_data['Data'].reset_index(drop=True)
    resultados['Valor_Real'] = y_test.reset_index(drop=True) 
    resultados['Valor_Previsto'] = y_pred

    data_especifica = pd.to_datetime("2023-09-27")
    previsao_especifica = resultados[resultados['Data'] == data_especifica]

    if not previsao_especifica.empty:
        print(previsao_especifica)
    else:
        print(f"Não há dados disponíveis para a data {data_especifica.date()}")


9/9 [==============================] - 0s 747us/step
MSE: 2.7919737252474057, MAE: 1.3473550301811008, R^2: -0.00690791228512988
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/euro.csv:
MSE: 2.7919737252474057
MAE: 1.3473550301811008
R^2: -0.00690791228512988

          Data  Valor_Real  Valor_Previsto
273 2023-09-27       -1.16        0.169619
12/12 [==============================] - 0s 725us/step
MSE: 2.7583911771564646, MAE: 1.3269640966237248, R^2: -0.00044430595116118354
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/dolar.csv:
MSE: 2.7583911771564646
MAE: 1.3269640966237248
R^2: -0.00044430595116118354

          Data  Valor_Real  Valor_Previsto
348 2023-09-27       -1.16         0.12227
8/8 [==============================] - 0s 856us/step
MSE: 54.666715415177435, MAE: 7.001767828484885, R^2: -19.948581855609877
Resultados para C:/Users/milen/OneDrive/Documentos/TCC/Bases/base_completa.csv:
MSE: 54.666715415177435
MAE: 7.001767828484885
R^2: -19.948581